In [ ]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
import csv


import folium
from folium import Map, Marker, Icon, Popup, PolyLine, CircleMarker
from folium.plugins import MarkerCluster
import branca
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from typing import Dict, Tuple, Optional

client = MongoClient("172.22.32.1", 27017) 
db = client.fit3182_db

In [ ]:
# ── Your starter connection ──
camera_coll = db.Camera
vehicle_coll = db.Vehicle

# ── Skip if already imported ──
if camera_coll.estimated_document_count() > 0:
    print("Camera collection already contains data. Skipping import.")
else:
    idx_name = camera_coll.create_index(
        [("pos", pymongo.ASCENDING)],
        name="pos_idx"
    )
    print(f"Ensured index on 'pos': {idx_name}")
    
    # ── Load CSV and insert ──
    csv_path = 'data/camera.csv'
    docs = []
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            docs.append({
                "_id":        int(row['camera_id']),
                "lat":        float(row['latitude']),
                "long":       float(row['longitude']),
                "pos":        float(row['position']),
                "speed_limit": int(row['speed_limit'])
            })

    if docs:
        result = camera_coll.insert_many(docs)
        print(f"Inserted {len(result.inserted_ids)} camera documents.")
        print("Current indexes on Camera:")
        for name, info in camera_coll.index_information().items():
            print(f" • {name}: {info['key']}")
    else:
        print("No camera records found in CSV.")
        
if vehicle_coll.estimated_document_count() > 0:
    print("Vehicle collection already contains data. Skipping import.")
else:
    # ── Prepare sets & counters ──
    existing_ids = set(vehicle_coll.distinct('_id'))
    seen_in_file = set()
    docs_to_insert = []
    update_count = 0
    added_count = 0

    csv_path = 'data/vehicle.csv'
    with open(csv_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            plate = row['car_plate']

            # Parse the incoming registration_date
            ts = row['registration_date'].rstrip("Z")
            reg_date = datetime.fromisoformat(ts)

            if plate in seen_in_file:
    #             print("Found duplicate:")
                # Plate already in DB → check whether to update
                existing = vehicle_coll.find_one(
                    {"_id": plate}
                )
    #             print(existing)
                if existing and reg_date > existing['registration_date']:
    #                 print("Registration is newer, updating...")
                    # Only update if the CSV date is newer
                    vehicle_coll.update_one(
                        {"_id": plate},
                        {"$set": {
                            "registration_date": reg_date,
                            "owner_name":        row['owner_name'],
                            "owner_addr":        row['owner_addr'],
                            "vehicle_type":      row['vehicle_type']
                        }}
                    )
                    update_count += 1
            else:
                seen_in_file.add(plate)
                # Brand-new plate → schedule for insert
                vehicle_coll.insert_one({
                    "_id":               plate,
                    "owner_name":        row['owner_name'],
                    "owner_addr":        row['owner_addr'],
                    "vehicle_type":      row['vehicle_type'],
                    "registration_date": reg_date
                })
                added_count += 1

    # ── Do the batch insert, if any ──
    if added_count > 0:
        print(f"Inserted {added_count} new vehicle documents.")
    else:
        print("No new vehicle records to insert.")

    # ── Report on any updates we made ──
    if update_count:
        print(f"Updated {update_count} existing vehicle document{'s' if update_count>1 else ''}.")

    # ── (Optional) show your indexes ──
    print("Current indexes on Vehicle:")
    for name, info in vehicle_coll.index_information().items():
        print(f" • {name}: {info['key']}")

In [ ]:


# Configuration constants\m
MONGO_HOST = "172.22.32.1"
MONGO_PORT = 27017
MONGO_DB   = "fit3182_db"
CAM_COLL   = "Camera"
VIOL_COLL  = "Violation"


# Data loading functions
def load_camera_locations() -> Dict[int, Tuple[float, float]]:
    """
    Load camera coordinates from MongoDB and return a mapping:
        camera_id -> (latitude, longitude)
    """
    client = MongoClient(host=MONGO_HOST, port=MONGO_PORT)
    coll = client[MONGO_DB][CAM_COLL]
    records = coll.find({}, {"_id": 1, "lat": 1, "long": 1})
    result = {rec['_id']: (rec['lat'], rec['long']) for rec in records}
    client.close()
    return result


def fetch_violations_for(
    date_str: str,
    hour: Optional[int] = None
) -> pd.DataFrame:
    """
    Query the violations collection for a specific date (YYYY-MM-DD) and optional hour.
    Returns a flat DataFrame of:
        car_plate, type, camera_id_start, camera_id_end,
        timestamp_start, timestamp_end, measured_speed
    """
    client = MongoClient(host=MONGO_HOST, port=MONGO_PORT)
    coll   = client[MONGO_DB][VIOL_COLL]
    
    start = datetime.strptime(date_str, "%Y-%m-%d")
    end   = datetime(start.year, start.month, start.day + 1)

    docs = list(coll.find({"date": {"$gte": start, "$lt": end}}))
    client.close()

    rows = []
    for doc in docs:
        plate = doc['car_plate']
        for v in doc.get('violations', []):
            rows.append({
                'car_plate': plate,
                'type': v['type'],
                'camera_id_start': v['camera_id_start'],
                'camera_id_end': v.get('camera_id_end'),
                'timestamp_start': v['timestamp_start'],
                'timestamp_end': v.get('timestamp_end'),
                'measured_speed': v['measured_speed']
            })

    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df['timestamp_start'] = pd.to_datetime(df['timestamp_start'])
    if hour is not None:
        df = df[df['timestamp_start'].dt.hour == hour]
    return df


# Map building helpers
def compute_map_center(
    df: pd.DataFrame,
    camera_locations: Dict[int, Tuple[float, float]]
) -> Tuple[float, float]:
    """Compute the geographic center based on unique start-camera locations."""
    coords = [camera_locations[cid]
              for cid in df['camera_id_start'].unique()
              if cid in camera_locations]
    latitudes, longitudes = zip(*coords)
    return (sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes))


def add_violation_clusters(
    m: Map,
    df: pd.DataFrame,
    camera_locations: Dict[int, Tuple[float, float]]
):
    """Add clustered markers for every violation on the map."""
    cluster = MarkerCluster(name='Violations').add_to(m)
    for _, row in df.iterrows():
        loc = camera_locations.get(row['camera_id_start'])
        if not loc:
            continue
        color = 'red' if 'VIOLATION' in row['type'].upper() else 'blue'
        time_val = (
            row['timestamp_start'] 
            if row['type'] == 'instantaneous' 
            else row['timestamp_end']
        )
        # normalize to a datetime object
        if isinstance(time_val, str):
            # remove trailing 'Z' if present and parse
            ts = time_val.rstrip('Z')
            dt_obj = datetime.fromisoformat(ts)
        else:
            dt_obj = time_val

        # format to HH:MM:SS
        time_str = dt_obj.strftime("%H:%M:%S")
        popup = (
            f"<b>Plate:</b> {row['car_plate']}<br>"
            f"<b>Type:</b> {row['type']}<br>"
            f"<b>Time:</b> {time_str}<br>"
            f"<b>Speed:</b> {row['measured_speed']}"
        )
        Marker(location=loc, popup=popup, icon=Icon(color=color))\
            .add_to(cluster)


def add_segment_polylines(
    m: Map,
    df: pd.DataFrame,
    camera_locations: Dict[int, Tuple[float, float]],
    colormap: branca.colormap.LinearColormap
):
    """Draw polylines between cameras for 'average' violations, colored by count."""
    seg = df[df['type'] == 'average'][['camera_id_start', 'camera_id_end']]
    if seg.empty:
        return
    counts = seg.groupby(['camera_id_start', 'camera_id_end']).size()\
                .reset_index(name='count')

    for _, row in counts.iterrows():
        a, b, cnt = row['camera_id_start'], row['camera_id_end'], row['count']
        loc_a = camera_locations.get(a)
        loc_b = camera_locations.get(b)
        if not loc_a or not loc_b:
            continue
        color = colormap(cnt)
        PolyLine(
            locations=[loc_a, loc_b],
            color=color,
            weight=4,
            opacity=0.8,
            popup=(f"<b>Camera {a}→{b}:</b><br>"
                   f"<b>{int(cnt)}</b> AVG Violations")
        ).add_to(m)


def add_camera_hotspots(
    m: Map,
    df: pd.DataFrame,
    camera_locations: Dict[int, Tuple[float, float]]
):
    """Overlay circle markers at cameras sized by total violation count."""
    seg = df[df['type'] == 'average'][['camera_id_start', 'camera_id_end']]
    if seg.empty:
        return
    combined = pd.concat([
        seg[['camera_id_start']].rename(columns={'camera_id_start': 'cam'}),
        seg[['camera_id_end']].rename(columns={'camera_id_end': 'cam'})
    ])
    totals = combined['cam'].value_counts().reset_index()
    totals.columns = ['cam', 'total']

    for _, row in totals.iterrows():
        cam, total = row['cam'], row['total']
        loc = camera_locations.get(cam)
        if not loc:
            continue
        CircleMarker(
            location=loc,
            radius=4 + total**0.5,
            color='orange',
            fill=True,
            fill_opacity=0.6,
            popup=(
                f"<b>Camera {cam}:</b><br>"
                f"<b>{int(total)}</b> Violations"
            )
        ).add_to(m)


def create_violations_map(
    date_str: str,
    hour: Optional[int],
    output: str = 'violations_map.html'
) -> folium.Map:
    """
    Generate a Folium map of violations for the given date/hour, saving to `output`.
    """
    camera_locations = load_camera_locations()
    df = fetch_violations_for(date_str, hour)
    if df.empty:
        print('No violations to plot.')
        return None

    center = compute_map_center(df, camera_locations)
    m = Map(location=center, zoom_start=13)

    # Color scale: fixed 0–20 (or dynamic based on data)
    colormap = branca.colormap.LinearColormap(
        ['green', 'yellow', 'red'], vmin=0, vmax=500,
        caption='Avg Violations per Segment'
    )
    colormap.add_to(m)

    add_violation_clusters(m, df, camera_locations)
    add_segment_polylines(m, df, camera_locations, colormap)
    add_camera_hotspots(m, df, camera_locations)

    m.get_root().html.add_child(
        folium.Element(f"<h3 style='text-align:center'>Violations on {date_str}" +
                      (f" at {hour:02d}:00" if hour is not None else '') +
                      '</h3>')
    )
    return m